In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
!pip install cplex -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.4 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [11]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [15]:
model = pyo.ConcreteModel()

#Sets
model.i = pyo.Set(initialize = ['PL1', 'PL2'])
model.j = pyo.Set(initialize = ['Type1', 'Type2', 'Type3'])

#Parameters
model.S = pyo.Param(model.i, model.j, initialize = ({('PL1', 'Type1'): 20, ('PL1', 'Type2'): 30, ('PL1', 'Type3'): 40,
                                                     ('PL2', 'Type1'): 50, ('PL2', 'Type2'): 35, ('PL2', 'Type3'): 45,
                                                     }))

S = model.S

model.PC = pyo.Param(model.i, initialize = {'PL1': 1000, 'PL2': 2000})
PC = model.PC

model.Sa = pyo.Param(model.i, initialize = {'PL1':400, 'PL2': 600})

Sa = model.Sa

model.x = pyo.Var(model.i, domain = pyo.NonNegativeIntegers)

x = model.x

model.y = pyo.Var(model.i, domain = pyo.Binary)

y = model.y

def Objective_rule(model):
  return sum(PC[i]*y[i] for i in model.i) + sum(Sa[i]*x[i] for i in model.i)

model.Obj = pyo.Objective(rule = Objective_rule, sense = pyo.minimize)

def Constraint1(model, j):
  return sum(S[i, 'Type1' ]*x[i] for i in model.i ) >= 120

model.Constr1 = pyo.Constraint(model.i, rule = Constraint1)

def Constraint2(model, j):
  return sum( sum(S[i, 'Type2']*x[i] for j in model.j) for i in model.i ) >= 150

model.Constr2 = pyo.Constraint(model.i, rule = Constraint2)

def Constraint3(model, j):
  return sum( sum(S[i, 'Type3']*x[i] for j in model.j) for i in model.i ) >= 200

model.Constr3 = pyo.Constraint(model.i, rule = Constraint3)

def Constraint4(model, i):
  return x[i] <= 30 *y[i]

model.Constr4 = pyo.Constraint(model.i, rule = Constraint4)

Solver = SolverFactory('cplex_direct')

results = Solver.solve(model, tee = True)

print(results)

print("Objective Function", model.Obj())

for i in model.i:
  print("Number of worker Employed in ", i, "is = ", x[i]())

Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 3400.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 0 columns.
MIP Presolve modified 17 coefficients.
Reduced MIP has 4 rows, 4 columns, and 8 nonzeros.
Reduced MIP has 2 binaries, 2 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve modified 2 coefficients.
Reduced MIP has 4 rows, 4 columns, and 8 nonzeros.
Reduced MIP has 2 binaries, 2 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.00 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 1.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.00 sec. (0.00 ticks)

        Nodes                                    